# Generating New-line delimited JSON of label metadata

In [ ]:
pip install beautifulsoup4

In [35]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [2]:
from google.cloud import storage
import json
import xml.etree.ElementTree as et

client = storage.Client()
bucket = client.bucket('cc-plan-a')

jsonitems = [] 
itemscount = 0
counter = 0

for blob in bucket.list_blobs(prefix='xml'):
    itemscount = itemscount + 1
    file = open("demo.xml", 'r+')
    blob.download_to_filename("demo.xml")
    tree = et.parse("demo.xml")
    root = tree.getroot()
    
    
    for obj in root.iter('filename'):
        filename = obj.text
     
    # width and height may be used for coordinate labeling adjustments later on
    for obj in root.iter('width'):
        width = obj.text
        
    for obj in root.iter('height'):
        height = obj.text
    
    #instantiate lists sheetnum and sheettile, sheet num always comes first so we can extract with list indexing after iterating
    label = []
    xmin = []
    xmax = []
    ymin = []
    ymax = []
    
    for obj in root.iter('name'):
        tag = obj.text
        label.append(tag)
        
    for obj in root.iter('xmin'):
        tag = obj.text
        xmin.append(tag)
        
    for obj in root.iter('xmax'):
        tag = obj.text
        xmax.append(tag)
        
    for obj in root.iter('ymin'):
        tag = obj.text
        ymin.append(tag)
        
    for obj in root.iter('ymax'):
        tag = obj.text
        ymax.append(tag)
              
    # accounting for edge cases, building dict of only sheet title and sheet num
    count = 0 #counter to ensure bouth sheetnum and sheet title exist
    
    for i in range(len(label)):
        if label[i] == "sheetnum":
            sheetnum_xmin = xmin[i]
            sheetnum_xmax = xmax[i]
            sheetnum_ymin = ymin[i]
            sheetnum_ymax = ymax[i]
            count = count + 1
        elif label[i] == "sheettitle":
            sheettitle_xmin = xmin[i]
            sheettitle_xmax = xmax[i]
            sheettitle_ymin = ymin[i]
            sheettitle_ymax = ymax[i]
            count = count + 1
        
    if count < 2:
        counter = counter + 1
        continue # don't add item if it doesn't have sheetnum and sheet title
        
        
    #Logic for putting label values on [0,1] scale as needed by GCP image detection schema
    snxmin = int(sheetnum_xmin) / int(width)
    snxmax = int(sheetnum_xmax) / int(width)
    snymin = int(sheetnum_ymin) / int(height)
    snymax = int(sheetnum_ymax) / int(height)
    
    stxmin = int(sheettitle_xmin) / int(width)
    stxmax = int(sheettitle_xmax) / int(width)
    stymin = int(sheettitle_ymin) / int(height)
    stymax = int(sheettitle_ymax) / int(height)
           
    #prepare bounding boxes into list form
    labels = [] 
    sheetnum = {
        "displayName": "sheetnum",
        "xMin": snxmin,
        "xMax": snxmax,
        "yMin": snymin,
        "yMax": snymax,
        "annotationResourceLabels": {"aiplatform.googleapis.com/annotation_set_name":"cc-plan-a-test_iod"}
    }
    
    labels.append(sheetnum)
    
    sheettitle = {
        "displayName": "sheettitle",
        "xMin": stxmin,
        "xMax": stxmax,
        "yMin": stymin,
        "yMax": stymax,
        "annotationResourceLabels": {"aiplatform.googleapis.com/annotation_set_name":"cc-plan-a-test_iod"}
    }
    labels.append(sheettitle)
    
    #create JSON object to be added to file   
    data = {
        "imageGcsUri": "gs://cc-plan-a/jpg/{}".format(filename),
        "boundingBoxAnnotations": labels,
        "dataItemResourceLabels": {}      
    }
    
    jsonitems.append(data) 
    
    file.truncate()
    file.close()
    
     # END OF OUTER FOR LOOP
        
print(counter)
labeljson = open('images_labeled.jsonl', 'w', encoding='utf=8')
for i in jsonitems:
    json.dump(i, labeljson)
    labeljson.write("\n")
    

bucket1 = client.get_bucket('cc-plan-a')
blob = bucket1.blob('images_labeled.jsonl')
blob.upload_from_filename('images_labeled.jsonl')
print("done")

1
done


## Extracting data labels for "Plan B" images

In [3]:
bucket2 = client.bucket('cc-plan-b')

jsonitems = [] 
itemscount = 0
for blob in bucket2.list_blobs(prefix='xml'):
    itemscount = itemscount + 1
    file = open("demo2.xml", 'r+')
    blob.download_to_filename("demo2.xml")
    tree = et.parse("demo2.xml")
    root = tree.getroot()
    
    for obj in root.iter('filename'):
        filename = obj.text
    
    
    # width and height may be used for coordinate labeling adjustments later on
    for obj in root.iter('width'):
        width = obj.text
        
    for obj in root.iter('height'):
        height = obj.text
    
    #instantiate lists sheetnum and sheettile, sheet num always comes first so we can extract with list indexing after iterating
    label = []
    xmin = []
    xmax = []
    ymin = []
    ymax = []
    
    
    for obj in root.iter('name'):
        tag = obj.text
        label.append(tag)
        
    for obj in root.iter('xmin'):
        tag = obj.text
        xmin.append(tag)
        
    for obj in root.iter('xmax'):
        tag = obj.text
        xmax.append(tag)
        
    for obj in root.iter('ymin'):
        tag = obj.text
        ymin.append(tag)
        
    for obj in root.iter('ymax'):
        tag = obj.text
        ymax.append(tag)
        
        
    # accounting for edge cases, building dict of only sheet title and sheet num
    count = 0 #counter to ensure bouth sheetnum and sheet title exist
    
    for i in range(len(label)):
        if label[i] == "sheetnum":
            sheetnum_xmin = xmin[i]
            sheetnum_xmax = xmax[i]
            sheetnum_ymin = ymin[i]
            sheetnum_ymax = ymax[i]
            count = count + 1
        elif label[i] == "sheettitle":
            sheettitle_xmin = xmin[i]
            sheettitle_xmax = xmax[i]
            sheettitle_ymin = ymin[i]
            sheettitle_ymax = ymax[i]
            count = count + 1
        
    if count < 2:
        continue # don't add item if it doesn't have sheetnum and sheet title
        
        
    #Logic for putting label values on [0,1] scale as needed by GCP image detection schema
    snxmin = int(sheetnum_xmin) / int(width)
    snxmax = int(sheetnum_xmax) / int(width)
    snymin = int(sheetnum_ymin) / int(height)
    snymax = int(sheetnum_ymax) / int(height)
    
    stxmin = int(sheettitle_xmin) / int(width)
    stxmax = int(sheettitle_xmax) / int(width)
    stymin = int(sheettitle_ymin) / int(height)
    stymax = int(sheettitle_ymax) / int(height)  
           
    #prepare bounding boxes into list form
    labels = [] 
    sheetnum = {
        "displayName": "sheetnum",
        "xMin": snxmin,
        "xMax": snxmax,
        "yMin": snymin,
        "yMax": snymax,
        "annotationResourceLabels": {"aiplatform.googleapis.com/annotation_set_name":"cc-plan-b-test_iod"}
    }
    
    labels.append(sheetnum)
   
    sheettitle = {
        "displayName": "sheettitle",
        "xMin": stxmin,
        "xMax": stxmax,
        "yMin": stymin,
        "yMax": stymax,
        "annotationResourceLabels": {"aiplatform.googleapis.com/annotation_set_name":"cc-plan-b-test_iod"}
    }
    labels.append(sheettitle)
    
    #create JSON object to be added to file   
    data = {
        "imageGcsUri": "gs://cc-plan-b/jpg/{}".format(filename),
        "boundingBoxAnnotations": labels,
        "dataItemResourceLabels": {}      
    }
    
    jsonitems.append(data) 
    
    file.truncate()
    file.close()
    
     # END OF OUTER FOR LOOP
        
labeljson = open('images_labeled-b.jsonl', 'w', encoding='utf=8')
for i in jsonitems:
    json.dump(i, labeljson)
    labeljson.write("\n")
    
bucket1 = client.get_bucket('cc-plan-b')
blob = bucket1.blob('images_labeled-b.jsonl')
blob.upload_from_filename('images_labeled-b.jsonl')
print("done")

done
